In [1]:
from security import Credentials
from firebase_admin import initialize_app
from firebase_admin.credentials import Certificate
from pipelines import Pipeline
from db import FirestoreDB
from services import NewsService, NLPService, SentimentService
from nlp import NLPModel, NLPTokenizer, NLPPostProcessor
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Trying to open credentials: [c:\Users\tomsr\Documents\School\aidi\student projects\crypto_bot\server/keys.json]


c:\Users\tomsr\Documents\School\aidi\student projects\crypto_bot\server\.ctb_server\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trying to open credentials: [c:\Users\tomsr\Documents\School\aidi\student projects\crypto_bot\server/keys.json]


In [2]:
app = initialize_app(Certificate(Credentials("firebase").load()))

Trying to open credentials: [c:\Users\tomsr\Documents\School\aidi\student projects\crypto_bot\server/keys.json]


In [3]:
db = FirestoreDB(app)
news_service = NewsService()
sentiment_service = SentimentService()
nlp_service =   NLPService(
    model = NLPModel(
        model=AutoModelForSequenceClassification.from_pretrained(
            "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
        ),
        is_auto_model=True,
    ),
    tokenizer = NLPTokenizer(
        tokenizer=AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
    ),
    postprocessor = NLPPostProcessor(chain=["set",])
)

In [4]:
context = {
    "db": db,
    "symbol": "BTC-USDT",
}

In [5]:
nlp_pipeline_sentiment_get = Pipeline(sentiment_service.get_earliest, "nlp_sentiment_get")

In [6]:
next_context = nlp_pipeline_sentiment_get.action(**context)

[<services.sentiment_service.SentimentService object at 0x000001FB847F3B90>] Running get function


In [7]:
nlp_pipeline_news_get = Pipeline(news_service.fetch_earliest, "nlp_news_get")

In [8]:
next_context = nlp_pipeline_news_get.action(**next_context)
next_context

[<services.news_service.NewsService object at 0x000001FB84EE3AD0>] Running get function


{'db': <db.firestore.FirestoreDB at 0x1fb84ef4610>,
 'symbol': 'BTC-USDT',
 'sentiment_batches': [{'__coll__': 'Batch_SENTIMENT',
   '__id__': 'de33c018-0104-456f-9710-fabf556cf785',
   '__db__': None,
   '__batch__': [{'__coll__': 'Sentiment',
     '__id__': 'b9de2651-9466-4cd4-8100-3186267992b2',
     '__db__': None,
     '__ts__': 1710264205,
     '__prediction__': 2.0},
    {'__coll__': 'Sentiment',
     '__id__': '99129927-67ba-4265-99d4-6d432538d3bc',
     '__db__': None,
     '__ts__': 1709228769,
     '__prediction__': 2.0},
    {'__coll__': 'Sentiment',
     '__id__': '33c25832-148b-41f0-bdad-f2f98a98e2c8',
     '__db__': None,
     '__ts__': 1710281636,
     '__prediction__': 2.0},
    {'__coll__': 'Sentiment',
     '__id__': 'd2e0d049-c92b-4772-b7e1-7e1d2a1d48d2',
     '__db__': None,
     '__ts__': 1709180400,
     '__prediction__': 1.0},
    {'__coll__': 'Sentiment',
     '__id__': '42e43904-b667-47c7-9186-1f204b2ebedc',
     '__db__': None,
     '__ts__': 1710363529,
    

In [7]:
nlp_pipeline_news_fetch = Pipeline(news_service.fetch, "nlp_news_fetch")

In [8]:
next_context = nlp_pipeline_news_fetch.action(**next_context)

[<services.news_service.NewsService object at 0x000002207F1DBBD0>] Running fetch function
[NLPDataFrame] Loading data from API Inputs


In [9]:
nlp_pipeline_news_tokenize = Pipeline(nlp_service.tokenize, "nlp_news_tokenizer")

In [10]:
next_context = nlp_pipeline_news_tokenize.action(**next_context)

[NLPTokenizer(device=cuda:0, tokenizer=RobertaTokenizerFast(name_or_path='mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
})]


Tokenizing: 100%|██████████| 100/100 [00:00<00:00, 1693.83it/s]


{'tensors': [{'input_ids': tensor([[    0,  9682, 14254,  3491,    68,  6232,   448,    31,  8518,  4582,
             1721,  1448, 14254,    17,    27,    29,  8518,    36, 38614,    43,
             4582,    33, 19057,    41, 29692,  1538,  1963,     9,    68,  6232,
              153,     6,    19,     5, 19081,    17,    27,    29,   923,  2284,
               30,  5913,   207,  1174,  5603,  2600,  1448, 14254,  3491,    68,
             6232,   448,    31,  8518,  4582, 50118,   133,   618,  1448, 14254,
             3491,    68,  6232,   448,    31,  8518,  4582,  1382,  1555,     2]],
          device='cuda:0'),
   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
          device='cuda:0')},
  {'input_ids': tensor([[    0,  9682, 14254,    18, 24134, 21416,    12, 237

In [11]:
nlp_pipeline_news_predict = Pipeline(nlp_service.predict, "nlp_news_predict")

In [12]:
next_context = nlp_pipeline_news_predict.action(**next_context)

[CryptoBotNLPModel(device=cuda:0, model=RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_

Predicting: 100%|██████████| 100/100 [00:02<00:00, 49.35it/s]


{'predictions': array([2., 2., 2., 1., 2., 2., 2., 0., 2., 1., 2., 2., 2., 0., 2., 2., 2.,
        2., 2., 2., 1., 2., 2., 2., 0., 2., 1., 1., 2., 1., 2., 2., 1., 1.,
        2., 2., 1., 2., 0., 2., 2., 0., 2., 2., 1., 1., 2., 2., 1., 2., 2.,
        1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 2., 0.,
        2., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 1., 1., 0., 0., 2.,
        2., 1., 0., 0., 2., 2., 2., 2., 2., 2., 1., 2., 0., 2., 2.]),
 'tensors': [{'input_ids': tensor([[    0,  9682, 14254,  3491,    68,  6232,   448,    31,  8518,  4582,
             1721,  1448, 14254,    17,    27,    29,  8518,    36, 38614,    43,
             4582,    33, 19057,    41, 29692,  1538,  1963,     9,    68,  6232,
              153,     6,    19,     5, 19081,    17,    27,    29,   923,  2284,
               30,  5913,   207,  1174,  5603,  2600,  1448, 14254,  3491,    68,
             6232,   448,    31,  8518,  4582, 50118,   133,   618,  1448, 14254,
             3491, 

In [13]:
nlp_pipeline_news_postprocess = Pipeline(nlp_service.postprocess, "nlp_news_postprocess")

In [14]:
next_context = nlp_pipeline_news_postprocess.action(**next_context)

{'tensors': [{'input_ids': tensor([[    0,  9682, 14254,  3491,    68,  6232,   448,    31,  8518,  4582,
             1721,  1448, 14254,    17,    27,    29,  8518,    36, 38614,    43,
             4582,    33, 19057,    41, 29692,  1538,  1963,     9,    68,  6232,
              153,     6,    19,     5, 19081,    17,    27,    29,   923,  2284,
               30,  5913,   207,  1174,  5603,  2600,  1448, 14254,  3491,    68,
             6232,   448,    31,  8518,  4582, 50118,   133,   618,  1448, 14254,
             3491,    68,  6232,   448,    31,  8518,  4582,  1382,  1555,     2]],
          device='cuda:0'),
   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
          device='cuda:0')},
  {'input_ids': tensor([[    0,  9682, 14254,    18, 24134, 21416,    12, 237

In [15]:
nlp_pipeline_news_write = Pipeline(sentiment_service.write, "nlp_sentiment_write")

In [16]:
next_context = nlp_pipeline_news_write.action(**next_context)
next_context

{'tensors': [{'input_ids': tensor([[    0,  9682, 14254,  3491,    68,  6232,   448,    31,  8518,  4582,
             1721,  1448, 14254,    17,    27,    29,  8518,    36, 38614,    43,
             4582,    33, 19057,    41, 29692,  1538,  1963,     9,    68,  6232,
              153,     6,    19,     5, 19081,    17,    27,    29,   923,  2284,
               30,  5913,   207,  1174,  5603,  2600,  1448, 14254,  3491,    68,
             6232,   448,    31,  8518,  4582, 50118,   133,   618,  1448, 14254,
             3491,    68,  6232,   448,    31,  8518,  4582,  1382,  1555,     2]],
          device='cuda:0'),
   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
          device='cuda:0')},
  {'input_ids': tensor([[    0,  9682, 14254,    18, 24134, 21416,    12, 237